In [1]:
import math
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [65]:
#on importe les données
reduced_test = pd.read_csv("data/reduced_test.csv")
reduced_truncated_data = pd.read_csv("data/reduced_truncated_data.csv")
accuracy_group_to_predict = pd.read_csv("data/accuracy_group_to_predict.csv")

In [66]:
#on ne garde que les colonnes communes, afin que le modèle puisse s'appliquer sur l'un puis sur l'autre
reduced_truncated_data = reduced_truncated_data[reduced_truncated_data.columns.intersection(reduced_test.columns)]
reduced_test = reduced_test[reduced_truncated_data.columns.intersection(reduced_test.columns)]

In [67]:
#On obtient Y contenant l'accuracy_group pour chaque ligne de reduced_truncated_data
Y = pd.merge(reduced_truncated_data, accuracy_group_to_predict, on='installation_id', how='outer')
Y = Y["accuracy_group_y"]

In [68]:
print(np.shape(Y))
print(np.shape(reduced_truncated_data))
print(np.shape(reduced_test))
print(np.shape(accuracy_group_to_predict))
Y.head()

(17690,)
(17690, 884)
(1000, 884)
(3302, 3)


0    3.0
1    3.0
2    3.0
3    3.0
4    3.0
Name: accuracy_group_y, dtype: float64

In [52]:
#il faut enlever les colonnes non numériques 
del reduced_truncated_data['accuracy_group']
#TODO à finir

In [53]:
model = LinearRegression().fit(reduced_truncated_data, Y)

ValueError: could not convert string to float: '0006a69f'

(17690, 883)